In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip install tensorflow==2.0

In [0]:
from sklearn.model_selection import KFold
import lightgbm

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn import preprocessing

import multiprocessing # 여러 개의 일꾼 (cpu)들에게 작업을 분산시키는 역할
from multiprocessing import Pool
from functools import partial # 함수가 받는 인자들 중 몇개를 고정 시켜서 새롭게 파생된 함수를 형성하는 역할
import os
import pandas as pd
import numpy as np
import joblib
import tensorflow as tf

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
tf.__version__

'2.0.0'

In [0]:
base = "drive/My Drive/외부/공모전/[기상] AI프렌즈 시즌1 온도 추정 경진대회/"

In [0]:
data = pd.read_csv(base + "train.csv")

In [0]:
for col in data.columns[data.max() == data.min()]:
  data = data.drop(columns=col)

In [0]:
data['hour'] = (data.index % 144 / 6).astype('int').astype('float')
data['minute'] = (data.index % 144 % 6).astype('int').astype('float')

In [0]:
X_cols = ['hour', 'X00', 'X07', 'X12', 'X20', 'X30', 'X31', 'X32', 'X37']
Y_cols = ["Y{0:02}".format(i) for i in range(18)]
# Y_cols = ['Y09', 'Y17']

In [0]:
train = pd.DataFrame(columns=X_cols + ['Y18'])

for Y in Y_cols:
  temp = data[X_cols + [Y]][:4320]
  temp.rename(columns={Y: 'Y18'}, inplace=True)

  train = pd.concat([train, temp])

val = data[X_cols + ['Y18']][4320:]

In [0]:
def pd2np(data):
  return np.asarray(data)
  # return np.expand_dims(np.asarray(data), -1)

In [0]:
x_train = pd2np(train[X_cols]).reshape(-1, 6, len(X_cols))
y_train = pd2np(train['Y18']).reshape(-1, 6)

x_val = pd2np(val[X_cols]).reshape(-1, 6, len(X_cols))
y_val = pd2np(val['Y18']).reshape(-1, 6)

In [0]:
x_train.shape, y_train.shape

((12960, 6, 9), (12960, 6))

In [0]:
def mse_AIFrenz(y_true, y_pred):
    diff = abs(y_true - y_pred)
    less_then_one = np.where(diff < 1, 0, diff)
    score = np.average(np.average(less_then_one ** 2, axis = 0))
    
    return score

In [0]:
def train_model(model, x_train=x_train, y_train=y_train):
  return model.fit(x_train, y_train)

def get_val_accuracy(model, x_val=x_val, y_val=y_val):
  pred = model.predict(x_val)
  mse = mse_AIFrenz(y_val, pred)

  return mse

def get_val_accuracy_sklearn(model, x_val=x_val, y_val=y_val):
  pred = model.predict(x_val)
  mse = mean_squared_error(y_val, pred)

  return mse

In [0]:
# from tensorflow.keras.optimizers import SGD
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.optimizers import Adadelta
# from tensorflow.keras.losses import categorical_crossentropy, mean_absolute_error, mean_squared_error
# from tensorflow.keras.layers import Activation, Dense, Dropout, Flatten, LeakyReLU

# with tf.device(tf.test.gpu_device_name()):
#   model = Sequential()

#   model.add(Dense(81, activation='tanh', input_shape = (len(X_cols),)))
#   model.add(Dropout(0.2))

#   model.add(Dense(729, activation='relu'))
#   model.add(Dropout(0.2))

#   model.add(Dense(243, activation='relu'))
#   model.add(Dropout(0.2))

#   model.add(Dense(729, activation='relu'))
#   model.add(Dropout(0.2))

#   model.add(Dense(243, activation='tanh'))
#   model.add(Dropout(0.2))

#   model.add(Dense(81, activation='relu'))

#   model.add(Dense(1, activation='linear'))

#   model.compile(optimizer=SGD(), loss=mean_squared_error, metrics=['mse'])
  
#   history = model.fit(x_train, y_train, validation_data=(x_val, y_val), 
#                       validation_freq=10, batch_size=1024, epochs=200, verbose=2, 
#                       use_multiprocessing=True)

In [0]:
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adadelta
from tensorflow.keras.losses import categorical_crossentropy, mean_absolute_error, mean_squared_error
from tensorflow.keras.layers import Input, Activation, Dense, Dropout, Flatten, LeakyReLU, Conv1D, BatchNormalization, Bidirectional, Flatten, GRU, LSTM

# with tf.device(tf.test.gpu_device_name()):
model = Sequential()

model.add(Conv1D(81, 3, padding='same', activation='tanh', input_shape = (6, len(X_cols))))
model.add(Conv1D(729, 3, padding='same', activation='relu'))
model.add(Conv1D(243, 3, padding='same', activation='relu'))

model.add(Conv1D(729, 3, padding='same', activation='relu'))

model.add(Conv1D(243, 3, padding='same', activation='tanh'))

model.add(Conv1D(81, 5, padding='same', activation='relu'))

model.add(Bidirectional(GRU(16)))

model.add(Dense(6, activation='linear'))

model.compile(optimizer=SGD(), loss=mean_squared_error, metrics=['mse'])

model.summary()

history = model.fit(x_train, y_train, validation_data=(x_val, y_val), 
                    validation_freq=10, batch_size=1024, epochs=200, verbose=2, 
                    use_multiprocessing=True)

Model: "sequential_49"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_149 (Conv1D)          (None, 6, 81)             2268      
_________________________________________________________________
conv1d_150 (Conv1D)          (None, 6, 729)            177876    
_________________________________________________________________
conv1d_151 (Conv1D)          (None, 6, 243)            531684    
_________________________________________________________________
conv1d_152 (Conv1D)          (None, 6, 729)            532170    
_________________________________________________________________
conv1d_153 (Conv1D)          (None, 6, 243)            531684    
_________________________________________________________________
conv1d_154 (Conv1D)          (None, 6, 81)             98496     
_________________________________________________________________
bidirectional_22 (Bidirectio (None, 32)              

In [0]:
hist = history
fig, loss_ax = plt.subplots(1,2, figsize=(12,5))

loss_ax[0].set_title("model loss")
loss_ax[1].set_title("model mse")

val_x_axis = [i * 10 for i in range(1, len(hist.history['val_mse']) + 1)]

loss_ax[0].plot(hist.history['loss'], 'y', label='train loss')
loss_ax[0].plot(val_x_axis, hist.history['val_loss'], 'r', label='val loss')
loss_ax[1].plot(hist.history['mse'], 'b', label='train mse')
loss_ax[1].plot(val_x_axis, hist.history['val_mse'], 'g', label='val mse')


loss_ax[0].set_xlabel('epoch')
loss_ax[1].set_xlabel('epoch')

loss_ax[0].set_ylabel('loss')
loss_ax[1].set_ylabel('mse')

loss_ax[0].legend(loc='upper left')
loss_ax[1].legend(loc='upper left')

plt.show()

In [0]:
print(get_val_accuracy(model))
print(get_val_accuracy_sklearn(model))

In [0]:
def submit(clf, X_cols, name="None"):
  submit = pd.read_csv(base + 'sample_submission.csv')
  test = pd.read_csv(base + "test.csv")
  test['hour'] = (test.index % 144 / 6).astype('float')
  test['minute'] = (test.index % 144 % 6).astype('float')

  pred = clf.predict(test[X_cols])

  submit['Y18'] = pred
  submit.to_csv(os.path.join(base, "제출", name + ".csv"), index=None)

In [0]:
# submit(model, X_cols, "dl")